In [9]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from pyjstat import pyjstat
import plotly.express as px

In [10]:
def get_eurostat_data(article_title, *args):
    '''
    To Do:
        - Write *args
        - Write _url_builder() helper func.
    '''
    
    article_title = article_title.upper()
    url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{article_title}?format=JSON&lang=EN'

    output = pyjstat.Dataset.read(url)
    df = output.write('dataframe')

    return df

In [72]:
df_crime = get_eurostat_data("crim_off_cat")

In [73]:
df_crime.loc[:, "Time"] = df_crime.loc[:, "Time"].astype(int)
df_crime

,Time frequency,International classification of crime for statistical purposes (ICCS),Unit of measure,Geopolitical entity (reporting),Time,value
0,Annual,Intentional homicide,Number,Belgium,2008,204.00
1,Annual,Intentional homicide,Number,Belgium,2009,189.00
2,Annual,Intentional homicide,Number,Belgium,2010,189.00
3,Annual,Intentional homicide,Number,Belgium,2011,214.00
4,Annual,Intentional homicide,Number,Belgium,2012,206.00
...,...,...,...,...,...,...
25825,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2018,0.11
25826,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2019,0.17
25827,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2020,0.28
25828,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2021,NaN


In [82]:

df_crime = df_crime.rename(columns={"Geopolitical entity (reporting)" : "country"})

df_crime.loc[df_crime.country == "Türkiye", "country"] = "Turkey"
df_crime["country"].unique()


array(['Belgium', 'Bulgaria', 'Czechia', 'Denmark', 'Germany', 'Estonia',
       'Ireland', 'Greece', 'Spain', 'France', 'Croatia', 'Italy',
       'Cyprus', 'Latvia', 'Lithuania', 'Luxembourg', 'Hungary', 'Malta',
       'Netherlands', 'Austria', 'Poland', 'Portugal', 'Romania',
       'Slovenia', 'Slovakia', 'Finland', 'Sweden', 'Iceland',
       'Liechtenstein', 'Norway', 'Switzerland', 'England and Wales',
       'Scotland (NUTS 2021)', 'Northern Ireland (UK) (NUTS 2021)',
       'Bosnia and Herzegovina', 'Montenegro', 'North Macedonia',
       'Albania', 'Serbia', 'Turkey', 'Kosovo*'], dtype=object)

In [83]:
countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden"
eu_countries = countries.split(", ")
eu_countries


['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden']

In [84]:
df_crime = df_crime[df_crime["country"].isin(eu_countries)]
df_homicide_be = df_crime[(df_crime.country == "Belgium") & (df_crime["Unit of measure"] == "Per hundred thousand inhabitants" ) & 
                                            (df_crime["International classification of crime for statistical purposes (ICCS)"] == "Intentional homicide")]





In [80]:
#get large database, clean it
df = get_eurostat_data("crim_off_cat")
df["Time"] = df["Time"].astype(int)

#get EU countries
countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden"
eu_countries = countries.split(", ")

#rename to country
df = df.rename(columns={"Geopolitical entity (reporting)" : "country"})

#change turkeys name
df.loc[df.country == "Türkiye", "country"] = "Turkey"

#include only EU countries
df = df[df["country"].isin(eu_countries)]


def get_crime_data(country, crime):
     return df[(df.country == country) & (df["Unit of measure"] == "Per hundred thousand inhabitants" ) & 
                                            (df["International classification of crime for statistical purposes (ICCS)"] == crime)]
    
    

In [89]:
crime_data = get_crime_data("Austria", "Intentional homicide")
print(crime_data.corr())
px.scatter(data_frame=crime_data, x="Time", y="value", color="country")


         Time   value
Time   1.0000 -0.1374
value -0.1374  1.0000
